## Metrics


In [1]:
import torchmetrics
from pathlib import Path

import datetime
from lib.dataloader import *


# Directory paths for semantic segmentation dataset
root_dir = Path("/home/emilia/WaterSegNet/datasets/complete_dataset/")
test_root = Path("/home/emilia/WaterSegNet/datasets/uav_dataset/")
dir_test_img = test_root / "test/images"
dir_test_mask = test_root / "test/labels"
dir_train_img = root_dir / "train/images"
dir_train_mask = root_dir / "train/labels"
dir_valid_img = root_dir / "valid/images"
dir_valid_mask = root_dir / "valid/labels"

# Directory paths for checkpoints and best models
dir_checkpoint = root_dir / "checkpoints/"
dir_best_model = root_dir / "best_models/"
dir_best_model /= datetime.datetime.now().strftime("%d-%m-%Y-%H-%M")


train_set = SegDataset(dir_train_img, dir_train_mask)
valid_set = SegDataset(dir_valid_img, dir_valid_mask)
test_set = SegDataset(dir_test_img, dir_test_mask)
data_module = SegDataModule(train_set, valid_set, test_set, batch_size=16)


metrics = torchmetrics.MetricCollection(
    torchmetrics.Accuracy(task="binary", num_classes=1, multiclass=False),
    torchmetrics.Recall(task="binary", num_classes=1, multiclass=False),
    torchmetrics.Precision(task="binary", num_classes=1, multiclass=False),
    torchmetrics.F1Score(
        task="binary", num_classes=1, multiclass=False
    ),  # Dice Coefficient
)

train_metrics = torchmetrics.MetricTracker(metrics)
val_metrics = torchmetrics.MetricTracker(metrics)
test_metrics = torchmetrics.MetricTracker(metrics)

## Models


In [2]:
import segmentation_models_pytorch as smp
from segnet.SegNet_model import *

Unet_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None,
)

Unet_plus_plus_model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None,
)

FPN_model = smp.FPN(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None,
)

SegNet_model = SegNet(
    n_channels=3,
    n_classes=1,
)

models = {
    "Unet": Unet_model,
    "Unetplusplus": Unet_plus_plus_model,
    "FPN": FPN_model,
    "SegNet": SegNet_model,
}

/home/emilia/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from lib.train import *
import segmentation_models_pytorch as smp
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer


model_name = "Unet"
model = models[model_name]

seg_model = SegModel(
    model,
    lr=1e-3,
    optimizer_type="adam",
    train_metrics=train_metrics,
    val_metrics=val_metrics,
    test_metrics=test_metrics,
    freeze_encoder=False,
)


checkpoints = {
    "Satelite": "/home/emilia/WaterSegNet/checkpoints/checkpoints_satelite/model-unet_resnet34_adam_padded_b16-v1.ckpt",
    "UAV": "/home/emilia/WaterSegNet/checkpoints/checkpoints_padded/model-unet_resnet34_adam_padded_b16.ckpt",
    "FULL": "/home/emilia/WaterSegNet/checkpoints/checkpoints_full/model-unet_resnet34_adam_padded_b16.ckpt",
}

for i in checkpoints:
    tb_logger = TensorBoardLogger("lightning_logs/test/", name=(model_name + "_" + i))

    trainer = Trainer(
        max_epochs=200,
        gpus=1 if torch.cuda.is_available() else 0,
        logger=tb_logger,
        precision=16,  # Mixed precision training
    )
    print(i)
    results = trainer.test(
        seg_model,
        ckpt_path=checkpoints[i],
        datamodule=data_module,
        verbose=False,
    )
    print(results)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/emilia/.local/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
Restoring states from the checkpoint path at /home/emilia/WaterSegNet/checkpoints/checkpoints_satelite/model-unet_resnet34_adam_padded_b16-v1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Satelite


Loaded model weights from checkpoint at /home/emilia/WaterSegNet/checkpoints/checkpoints_satelite/model-unet_resnet34_adam_padded_b16-v1.ckpt


Testing: 100%|██████████| 55/55 [00:03<00:00, 15.26it/s]

/home/emilia/.local/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Restoring states from the checkpoint path at /home/emilia/WaterSegNet/checkpoints/checkpoints_padded/model-unet_resnet34_adam_padded_b16.ckpt


...

Testing: 100%|██████████| 55/55 [00:03<00:00, 15.06it/s]

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Restoring states from the checkpoint path at /home/emilia/WaterSegNet/checkpoints/checkpoints_full/model-unet_resnet34_adam_padded_b16.ckpt



[{'test_loss': 0.0818043202161789, 'test.Accuracy': 0.6340163946151733, 'test.Recall': 0.6340163946151733, 'test.Precision': 0.6330986022949219, 'test.F1Score': 0.6335493326187134}]
FULL


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/emilia/WaterSegNet/checkpoints/checkpoints_full/model-unet_resnet34_adam_padded_b16.ckpt


Testing: 100%|██████████| 55/55 [00:03<00:00, 15.45it/s]
[{'test_loss': 0.09656822681427002, 'test.Accuracy': 0.6935898661613464, 'test.Recall': 0.6935898661613464, 'test.Precision': 0.6894514560699463, 'test.F1Score': 0.6914964914321899}]
